In [1]:
import requests
import datetime
import logging 
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
import os
import requests
import psycopg2 as psy


In [2]:
#Access virtual environment for sensitive details
load_dotenv()
api_key= os.getenv('WEATHER_API')


base_url= 'http://api.weatherstack.com/current'

locations= ['Edmonton', 'Calgary', 'Toronto', 'Vancouver', 'Winnipeg']

In [3]:
# Using API to access data 
weather_data=[]
for location in locations:
    params= {
    'access_key': api_key,
    'query': location
}
    response= requests.get(base_url, params= params)
    
    data= response.json()
    weather_data.append(data)

In [4]:
weather_data

[{'request': {'type': 'City',
   'query': 'Edmonton, Canada',
   'language': 'en',
   'unit': 'm'},
  'location': {'name': 'Edmonton',
   'country': 'Canada',
   'region': 'Alberta',
   'lat': '53.550',
   'lon': '-113.500',
   'timezone_id': 'America/Edmonton',
   'localtime': '2024-06-27 17:21',
   'localtime_epoch': 1719508860,
   'utc_offset': '-6.0'},
  'current': {'observation_time': '11:21 PM',
   'temperature': 15,
   'weather_code': 113,
   'weather_icons': ['https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0001_sunny.png'],
   'weather_descriptions': ['Sunny'],
   'wind_speed': 19,
   'wind_degree': 50,
   'wind_dir': 'NE',
   'pressure': 1005,
   'precip': 0.6,
   'humidity': 93,
   'cloudcover': 0,
   'feelslike': 13,
   'uv_index': 3,
   'visibility': 10,
   'is_day': 'yes'}},
 {'request': {'type': 'City',
   'query': 'Calgary, Canada',
   'language': 'en',
   'unit': 'm'},
  'location': {'name': 'Calgary',
   'country': 'Canada',
   'region': 'Alberta',

In [5]:
# We need to extract the relevant data from the response
# and convert it into a DataFrame

# Extract the relevant data
cleaned_data = []
for item in weather_data:
    location = item["location"]["name"]
    country = item["location"]["country"]
    temperature = item["current"]["temperature"]
    temp_feels_like= item["current"]["feelslike"]
    wind_speed = item["current"]["wind_speed"]
    pressure = item["current"]["pressure"]
    visibility= item["current"]["visibility"]
    humidity = item["current"]["humidity"]
    local_time= item["location"]['localtime']
    observation_time = item["current"]["observation_time"]

     # Convert the observation time to timestamp using today's date for the year, month and day
    observation_time = datetime.strptime(observation_time, "%I:%M %p").time()
    local_time = datetime.strptime(local_time, "%Y-%m-%d %H:%M")

    cleaned_data.append([location, country, temperature,temp_feels_like, wind_speed, pressure,visibility, humidity,local_time, observation_time])
    

In [6]:
# Convert the data into a DataFrame
columns = ["location", "country", "temperature", "temp_feels_like","wind_speed", "pressure", "visibility", "humidity","local_time", "observation_time"]
df = pd.DataFrame(cleaned_data, columns=columns)


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   location          5 non-null      object        
 1   country           5 non-null      object        
 2   temperature       5 non-null      int64         
 3   temp_feels_like   5 non-null      int64         
 4   wind_speed        5 non-null      int64         
 5   pressure          5 non-null      int64         
 6   visibility        5 non-null      int64         
 7   humidity          5 non-null      int64         
 8   local_time        5 non-null      datetime64[ns]
 9   observation_time  5 non-null      object        
dtypes: datetime64[ns](1), int64(6), object(3)
memory usage: 532.0+ bytes


In [8]:
#Creating connection
def get_connection():
    connection= psy.connect(
        dbname= os.getenv('DB_NAME'),
        user= os.getenv('DB_USER'),
        host=os.getenv('DB_HOST'),
        password= os.getenv('PASSWORD'),
        port= os.getenv('DB_PORT')     
)
    return connection
    
conn=get_connection()

#Create a cursor
cur=conn.cursor()

In [9]:
df.head()

,location,country,temperature,temp_feels_like,wind_speed,pressure,visibility,humidity,local_time,observation_time
0,Edmonton,Canada,15,13,19,1005,10,93,2024-06-27 17:21:00,23:21:00
1,Calgary,Canada,13,12,19,1003,14,82,2024-06-27 17:21:00,23:21:00
2,Toronto,Canada,21,21,20,1014,14,33,2024-06-27 19:19:00,23:19:00
3,Vancouver,Canada,14,14,4,1016,16,94,2024-06-27 16:21:00,23:21:00
4,Winnipeg,Canada,21,21,4,1007,10,46,2024-06-27 18:21:00,23:21:00


In [10]:
#Create table
conn=get_connection()
cur=conn.cursor()

create_table_query= '''
                DROP TABLE IF EXISTS weather;

                CREATE TABLE IF NOT EXISTS weather (
                id SERIAL PRIMARY KEY,
                location VARCHAR(255) NOT NULL,
                country VARCHAR(255) NOT NULL,
                temperature DECIMAL NOT NULL,
                temp_feels_like DECIMAL NOT NULL,
                wind_speed DECIMAL NOT NULL,
                pressure DECIMAL NOT NULL,
                visibility DECIMAL NOT NULL,
                humidity DECIMAL NOT NULL,
                local_time TIMESTAMP NOT NULL,
                observation_time TIME NOT NULL
                );     
'''

cur.execute(create_table_query)

conn.commit()
conn.close()

In [11]:
data_to_insert = df.values.tolist()
data_to_insert

[['Edmonton',
  'Canada',
  15,
  13,
  19,
  1005,
  10,
  93,
  Timestamp('2024-06-27 17:21:00'),
  datetime.time(23, 21)],
 ['Calgary',
  'Canada',
  13,
  12,
  19,
  1003,
  14,
  82,
  Timestamp('2024-06-27 17:21:00'),
  datetime.time(23, 21)],
 ['Toronto',
  'Canada',
  21,
  21,
  20,
  1014,
  14,
  33,
  Timestamp('2024-06-27 19:19:00'),
  datetime.time(23, 19)],
 ['Vancouver',
  'Canada',
  14,
  14,
  4,
  1016,
  16,
  94,
  Timestamp('2024-06-27 16:21:00'),
  datetime.time(23, 21)],
 ['Winnipeg',
  'Canada',
  21,
  21,
  4,
  1007,
  10,
  46,
  Timestamp('2024-06-27 18:21:00'),
  datetime.time(23, 21)]]

In [12]:
# Insert the df data into the databas
conn=get_connection()
cur=conn.cursor()

cur.executemany("""
        INSERT INTO weather (location, country, temperature, temp_feels_like, wind_speed, pressure, visibility,humidity, local_time, observation_time)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, data_to_insert)

conn.commit()
conn.close()